# Imports

In [1]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

import logging
import traceback
from pprint import pformat

from geopy.distance import geodesic

import numpy as np
from sklearn.cluster import DBSCAN

## Parameters

In [2]:
creds_path: str = "./keys/serviceAccountKey.json"
max_distance_km: float = 5.0

## Setup

In [3]:
logging.basicConfig(
    level=logging.INFO, format="%(levelname)s: %(asctime)s - %(message)s"
)

logger = logging.getLogger(__name__)

## Authentication

In [4]:
try:
    cred = credentials.Certificate(creds_path)
    logger.info("Firebase centantial read successfully")
    firebase_admin.initialize_app(cred)
    logger.info("Firebase initialized successfully")
    db = firestore.client()
    logger.info("Firestore client created")

except Exception as e:
    traceback_str = traceback.format_exc()
    logger.error("An error occurred: %s", str(e))
    logger.debug(f"Traceback: {traceback_str}")

INFO: 2024-01-19 21:53:58,612 - Firebase centantial read successfully
INFO: 2024-01-19 21:53:58,613 - Firebase initialized successfully
INFO: 2024-01-19 21:53:58,614 - Firestore client created


## Custom Error

In [5]:
class MarkerNotFoundError(Exception):
    pass

## Utils

In [6]:
def getAllMarkers(firebase_clent):
    markers = []

    try:
        markers_get = firebase_clent.collection("Markers").get()
        logger.info("Successfully retrived all Markers")

    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error("An error occurred: %s", str(e))
        logger.debug(f"Traceback: {traceback_str}")

    for m in markers_get:
        marker = {
            "marker-id": str(m.id),
            "marker_cord": (m.to_dict()["lat"], m.to_dict()["long"]),
        }
        markers.append(marker)

    logger.info("Successfully got all Markers into python dict")
    return markers

In [7]:
def find_markers_within_distance(
    markers: list, target_coord: tuple, max_distance_km: float = 5.0
):
    nearby_markers = []

    try:
        for marker in markers:
            marker_coord = marker["marker_cord"]
            distance = geodesic(target_coord, marker_coord).kilometers

            if distance <= max_distance_km:
                nearby_markers.append(marker)
        logger.debug(
            "Successfully looped over all markers and selected markers within distance"
        )
    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error("An error occurred: %s", str(e))
        logger.debug(f"Traceback: {traceback_str}")

    logger.info(f"Successfully got nearby markers: {len(nearby_markers)}")
    return nearby_markers

In [8]:
def cluster_markers_fn(markers, epsilon=1.0, min_samples=2):
    """
    Cluster markers by coordinates using DBSCAN.

    Parameters:
    - markers: List of dictionaries, each containing 'marker-id' and 'marker_cord'.
    - epsilon: Maximum distance between two samples for one to be considered in the neighborhood of the other.
    - min_samples: The number of samples in a neighborhood for a point to be considered a core point.

    Returns:
    - A modified list of markers with an additional 'cluster_label' field.
    """
    # Extract coordinates from markers
    coordinates = np.array([marker["marker_cord"] for marker in markers])

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    # Add cluster labels to the markers
    for i, marker in enumerate(markers):
        marker["cluster_label"] = labels[i]

    return markers

In [14]:
def get_cluster_label(marker_data, target_marker_id):
    for marker in marker_data:
        if marker['marker-id'] == target_marker_id:
            return marker['cluster_label']
    raise MarkerNotFoundError(f"Marker ID {target_marker_id} not found in the list.")

In [19]:
def upadteClusterIDfirestore(firebaseClient, clusters: list[dict]):
    marker_ids = [marker["marker-id"] for marker in clusters]
    try:
        for key in marker_ids:
            ref = firebaseClient.collection("Markers").document(key)
            ref.update(
                {
                    "cluster": int(get_cluster_label(clusters, key)),
                }
            )
        logger.info("Added updated cluster ids")
    except Exception as e:
        traceback_str = traceback.format_exc()
        logger.error("An error occurred: %s", str(e))
        logger.debug(f"Traceback: {traceback_str}")

## Program

In [11]:
markers_data = getAllMarkers(db)
logger.info(f"Total markers retrived {len(markers_data)}")
logger.debug(pformat(markers_data))

INFO: 2024-01-19 21:54:27,200 - Successfully retrived all Markers
INFO: 2024-01-19 21:54:27,202 - Successfully got all Markers into python dict
INFO: 2024-01-19 21:54:27,203 - Total markers retrived 33


In [12]:
clustered_markers = cluster_markers_fn(markers_data, epsilon=0.1, min_samples=1)

In [ ]:
upadteClusterIDfirestore(firebaseClient=db, clusters=clustered_markers)

## testing

In [21]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(india_map)

# Add markers to the map
for marker in clustered_markers:
    folium.Marker(
        location=marker["marker_cord"],
        popup=f"Marker ID: {marker['marker-id']}, Cluster: {marker['cluster_label']}",
    ).add_to(marker_cluster)

# Save the map to an HTML file or display it
india_map

## class

In [14]:
class NearYou:
    def __init__(self, firebase_clent, max_distance_km: float) -> None:
        self.getAllMarkers(firebase_clent)
        self.max_distance_km = max_distance_km
        pass

    def getAllMarkers(self, firebase_clent) -> None:
        markers = []

        try:
            markers_get = firebase_clent.collection("Markers").get()
            logger.info("Successfully retrived all Markers")

        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

        for m in markers_get:
            marker = {
                "marker-id": m.to_dict()["id"],
                "marker_cord": (m.to_dict()["lat"], m.to_dict()["long"]),
            }
            markers.append(marker)

        logger.info("Successfully got all Markers into python dict")

        self.markers = markers

    def find_markers_within_distance(self, target_coord: tuple):
        nearby_markers = []

        try:
            for marker in self.markers:
                marker_coord = marker["marker_cord"]
                distance = geodesic(target_coord, marker_coord).kilometers

                if distance <= self.max_distance_km:
                    nearby_markers.append(marker)
            logger.debug(
                "Successfully looped over all markers and selected markers within distance"
            )
        except Exception as e:
            traceback_str = traceback.format_exc()
            logger.error("An error occurred: %s", str(e))
            logger.debug(f"Traceback: {traceback_str}")

        return nearby_markers